# Import Necessary packages

In [1]:
#!pip install neo4j
import re
import os
import getpass
import openai
from typing import Any, Dict, List
from openai import OpenAI
from neo4j import GraphDatabase

# Prerequisite set up

### Set up OpenAI key

In [2]:
name = 'OPENAI_API_KEY'
secret = getpass.getpass("Enter the OpenAI API Key")
os.environ[name] = secret

# Define your API key (keep it secret!)
openai.api_key = os.environ[name]
API_KEY = os.environ[name]

### Open a OpenAI session

In [3]:
client = OpenAI()

### Connect to the Cloud Neo4j graph database

In [4]:
# Connect to the Neo4j database
uri = "neo4j+s://28e5b287.databases.neo4j.io"  # Change to match your Neo4j server settings
username = "neo4j"      # Change to your Neo4j username
password = "z9J3DPCCxGYELn99XDdFbFIBnWOwR5fdn4MiG_Nvdck"      # Change to your Neo4j password

# Create a Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))

# Customized Langchain

### Revine users' input prompt

In [5]:
def refine_prompt(raw_prompt, openai_api_key=os.environ['OPENAI_API_KEY']):
    """
    Refines a raw user input using OpenAI's language model to create a more effective prompt.

    Parameters:
    raw_prompt (str): The original user input.
    openai_api_key (str): Your OpenAI API key.

    Returns:
    dict: A dictionary containing the status ('success' or 'failed') and the refined prompt.
    """
    try:
        openai.api_key = openai_api_key

        # Constructing the prompt for the LLM to refine the user input
        prompt_to_refine = (
            "The following is a plain English user input for querying a graph database: '{}'. "
            "Rewrite this input into a clearer query format. "
            "However, do not remove any information related to the database or information related to node type which the user provided. If the input is not clear or relevant, indicate that the user should provide a clearer query. "
            "Do not remove any information related to database and node attributes."
            "For example, if the input is 'I want to know how many nodes are in this database', "
            "a better format would be 'Find the number of nodes in the database'.\n"
            "Refined Input: ".format(raw_prompt)
        )

        # Limit the length of users' input 
        max_tokens = 100
        if len(raw_prompt.split()) > 100:
            max_tokens = len(raw_prompt.split())

        # Model set up to refine the users' input 
        response = client.completions.create(
            model="text-davinci-003",  # Use the latest available engine
            prompt=prompt_to_refine,
            max_tokens=100  # Adjust based on your needs
        )

        # refined prompt
        refined_prompt = response.choices[0].text.strip()

        # Check if the refined prompt is valid
        if "provide a clearer query" in refined_prompt.lower():
            return {"status": "failed", "updated_user_prompt": refined_prompt}
        else:
            return {"status": "success", "updated_user_prompt": refined_prompt}
    except Exception as e:
        print(f"Error in refining prompt: {e}")
        return {"status": "failed", "updated_user_prompt": raw_prompt}

### Help Function: Refined the output

In [6]:

def neo4j_query(query: str, params: dict = {}) -> List[Dict[str, Any]]:
    """
    Query Neo4j database.

    Args:
        query (str): Cypher query to be executed.
        params (dict, optional): Parameters for the Cypher query. Defaults to {}.

    Returns:
        List[Dict[str, Any]]: List of dictionaries representing the query results.

    Raises:
        ValueError: If the Cypher syntax is incorrect.
    """
    from neo4j.exceptions import CypherSyntaxError

    with driver.session(database="neo4j") as session:
        try:
            data = session.run(query, params)
            return [r.data() for r in data]
        except CypherSyntaxError as e:
            raise ValueError(f"Generated Cypher Statement is not valid\n{e}")

def refresh_schema() -> str:
    """
    Refreshes the Neo4j graph schema information and returns a string representation.

    Returns:
        str: String representation of the current schema.
    """

    node_properties_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
    WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
    RETURN {labels: nodeLabels, properties: properties} AS output
    """

    rel_properties_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
    WITH label AS nodeLabels, collect({property:property, type:type}) AS properties
    RETURN {type: nodeLabels, properties: properties} AS output
    """

    rel_query = """
    CALL apoc.meta.data()
    YIELD label, other, elementType, type, property
    WHERE type = "RELATIONSHIP" AND elementType = "node"
    UNWIND other AS other_node
    RETURN {start: label, type: property, end: toString(other_node)} AS output
    """

    node_properties = [el["output"] for el in neo4j_query(node_properties_query)]
    rel_properties = [el["output"] for el in neo4j_query(rel_properties_query)]
    relationships = [el["output"] for el in neo4j_query(rel_query)]

    schema_str = f"""
    Node properties are the following:
    {node_properties}
    Relationship properties are the following:
    {rel_properties}
    The relationships are the following:
    {[f"(:{el['start']})-[:{el['type']}]->(:{el['end']})" for el in relationships]}
    """

    return schema_str

def get_gpt_cypher_query(curr_schema:str, question:str,model="gpt-4",history=None):
    """
    Send a request to the OpenAI Chat API and get a response from the model.

    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        model (str): The model version to use, default is "gpt-3.5-turbo".
        history(dict): The historical dialogue between users and systems
    Returns:
        str: The model's response.
    """

    # Input the info of database with pre-dialogue
    system_prompt =  """

    "Human: Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided
    relationship types and properties in the schema.\nDo not use any other relationship types or
    properties that are not provided.\n
    Consider directionality of the graph.\n
    The cypher output should have some indication either as variable name to indicate the requirement of the
    question.\n
    Do not include any explanations or apologies in your responses.\n
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    \nSchema:
    {}".
    """.format(curr_schema)

    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    if history:
        messages.extend(history)

    ## Use the OpenAI Python client to send the request
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )

    # Extract the response text and return
    return response.choices[0].message.content


def get_gpt_user_output(prompt:str, question:str, model="gpt-4",history=None):
    """
    Send a request to the OpenAI Chat API and get a response from the model.

    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        model (str): The model version to use, default is "gpt-3.5-turbo".
        
    Returns:
        str: The model's response, which is the refined users' output. 
    """

    system_prompt =  """

    "Human: You are an assistant that helps to form nice and human understandable answers
    .\nThe information part contains the provided information that you must use to construct an answer.
    \nThe provided information is authoritative, you must never doubt it or try to use your internal knowledge
    to correct it.\nMake the answer sound as a response to the question.
    Do not mention that you based the result on the given information.\n
    Only If the information is empty then return whatever asked in the question does not exist\n
    Output only the node ids to represent the nodes, unless specified
    \nInformation:\n{}"
    """.format(prompt)

    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    if history:
        messages.extend(history)

    ## Use the OpenAI Python client to send the request
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )

    # Extract the response text and return
    return response.choices[0].message.content


### Main function for customized langchain

In [11]:
def lang_chain_custom(question:str,driver,model="gpt-4"):
    """
    Orchestrates generating a Cypher query using GPT-3.5, executing it on Neo4j, and formatting the response.

    Args:
        question (str): User's question.
        driver: Neo4j database driver.
        model (str, optional): Model version of GPT-3.5. Defaults to "gpt-3.5-turbo".

    Returns:
        str: Final response after processing the query and results.
    """
    curr_schema = refresh_schema()

    ## Getting Cypher query
    response = get_gpt_cypher_query(curr_schema, question, model)
    print('Cypher Query is',response)
    print('\n')

    ## Interacting with neo4j
    try:
        cypher_response = neo4j_query(response)
        cypher_response = cypher_response[:10] ## Limiting it to avoid token limits
        print('Neo4j answer is',cypher_response)
        print('\n')

    except Exception as e:
        try:
            print('Retrying')

            history = [
                    {"role": "assistant", "content": response},
                    {
                        "role": "user",
                        "content": f"""This query returns an error: {str(e)}
                        Give me a improved query that works without any explanations or apologies""",
                    },

            ]
            # get cypher query based on the question
            response = get_gpt_cypher_query(curr_schema, question,model,history)
            print('New Cypher Query is',response)
            print('\n')

            # parsing neo4j query from the cypher query generated from chatgpt
            cypher_response = neo4j_query(response)
            
            # Return the first 10 answers
            cypher_response = cypher_response[:10] ## Limiting it to avoid token limits
            print('Neo4j answer is',cypher_response)
            print('\n')

        except Exception as e:
            cypher_response = []

    ## Obtaining final answer
    response_2 = get_gpt_user_output(cypher_response, question, model)

    pattern = r'\bsorry\b'
    # re.IGNORECASE flag makes the search case-insensitive
    if(bool(re.search(pattern, response_2, re.IGNORECASE))):
    
        print('Retrying output string')

        history = [
        {"role": "assistant", "content": response_2},
        {
            "role": "user",
            "content": f"""This query returns an incomplete answer.
            If the information is empty then return does not exist or unknown.\n
            If not, assume the output is a simplified answer to
            the question {question}
            Give me a improved query that works without any explanations or apologies""",
        }]

        response_2 = get_gpt_user_output(cypher_response, question,model,history)

    return response_2

# Results

### Path existence

In [12]:
question = """
Does a path exist from node 56203 to node 38803 for nodes of type node? If so what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print(question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Find if there exists a path from node 56203 to node 38803 for nodes of type node, and if so, provide the path.
-----
Cypher Query is MATCH path = (start:Node_201511060000 {id:"56203"})-[:CONNECTS_201511060000*]-(end:Node_201511060000 {id:"38803"})
RETURN path


Neo4j answer is []


Retrying output string
The existence of a path from node 56203 to node 38803 is unknown.


In [13]:
question = """
Does a path exist from node 56203 to node 38803 for nodes of type node? If so what is the path with full node properties.
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the path from node 56203 to node 38803 for nodes of type 'node' and list the full node properties of the path.
-----
Cypher Query is MATCH p=(:Node_201511060000 {id: '56203'})-[:CONNECTS_201511060000*]->(:Node_201511060000 {id: '38803'})
RETURN nodes(p)


Neo4j answer is []


Retrying output string
The path from node 56203 to node 38803 for nodes of type 'node' does not exist.


In [14]:
question = """
Does a path exist from node 3356 to node 7018 for nodes of type node_2? If so what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the path from node 3356 to node 7018 for nodes of type node_2.
-----
Cypher Query is MATCH path=(:Node_201511060000 {id: '3356'})-[:CONNECTS_201511060000*]->(:Node_201511060000 {id: '7018'})
RETURN path


Neo4j answer is [{'path': [{'id': '3356'}, 'CONNECTS_201511060000', {'id': '2914'}, 'CONNECTS_201511060000', {'id': '3257'}, 'CONNECTS_201511060000', {'id': '2914'}, 'CONNECTS_201511060000', {'id': '3356'}, 'CONNECTS_201511060000', {'id': '3257'}, 'CONNECTS_201511060000', {'id': '3356'}, 'CONNECTS_201511060000', {'id': '3549'}, 'CONNECTS_201511060000', {'id': '3356'}, 'CONNECTS_201511060000', {'id': '7018'}]}, {'path': [{'id': '3356'}, 'CONNECTS_201511060000', {'id': '2914'}, 'CONNECTS_201511060000', {'id': '3257'}, 'CONNECTS_201511060000', {'id': '2914'}, 'CONNECTS_201511060000', {'id': '3356'}, 'CONNECTS_201511060000', {'id': '3257'}, 'CONNECTS_201511060000', {'id': '3356'}, 'CONNECTS_201511060000', {'id': '7018'}]}, {'path': [{'id': '3356'}, 'CONNECTS_20151

In [15]:
question = """
Does a path exist from node 4777 to node 7671 for nodes of type node_2? If so what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the path from node 4777 to node 7671 for nodes of type node_2.
-----
Cypher Query is MATCH path = (n:Node_201511060000 {id: '4777'})-[:CONNECTS_201511060000*]->(m:Node_201511060000 {id: '7671'})
RETURN path


Neo4j answer is []


Retrying output string
The path from node 4777 to node 7671 does not exist.


In [16]:
question = """
Does a path exist from node 123 to node 7671 for nodes of type node_2? If so what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the path from node 123 to node 7671 for nodes of type node_2.
-----
Cypher Query is MATCH path = (a:Node_201511060000{id:'123'})-[:CONNECTS_201511060000*]->(b:Node_201511060000{id:'7671'}) RETURN path


Neo4j answer is []


Retrying output string
The path from node 123 to node 7671 for nodes of type node_2 does not exist.


In [17]:
question = """
How many path exists from node 56203 to node 38803 for nodes of type node?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the number of paths between node 56203 and node 38803 for nodes of type node.
-----
Cypher Query is MATCH path = (:Node_201511060000 {id: '56203'})-[:CONNECTS_201511060000*]->(:Node_201511060000 {id: '38803'})
RETURN COUNT(path) AS NumberOfPaths


Neo4j answer is [{'NumberOfPaths': 0}]


There are no paths between node 56203 and node 38803 for nodes of type node.


In [18]:
question = """
How many path exists from node 3356 to node 7018 for nodes of type node?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the number of paths from node 3356 to node 7018 that traverse nodes of type 'node'.
-----
Cypher Query is MATCH p=(n:Node_201511060000{id:'3356'})-[:CONNECTS_201511060000*]->(m:Node_201511060000{id:'7018'})
RETURN count(p) as number_of_paths


Neo4j answer is [{'number_of_paths': 9451}]


There are 9451 paths from node 3356 to node 7018 that traverse nodes of type 'node'.


In [19]:
question = """
How many path exists from node 56203 to node 38803 for nodes of type node_2?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the number of paths from node 56203 to node 38803 for nodes of type node_2.
-----
Cypher Query is MATCH path=(n:Node_201511060000{id: '56203'})-[:CONNECTS_201511060000*]->(m:Node_201511060000{id: '38803'})
RETURN count(path) AS NumberOfPaths


Neo4j answer is [{'NumberOfPaths': 0}]


There are no paths from node 56203 to node 38803 for nodes of type node_2.


In [20]:
question = """
How many path exists from node 2519 to node 10026 for nodes of type node_2? What are they?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the number of paths from node 2519 to node 10026 of type node_2 and identify the specific paths.
-----
Cypher Query is MATCH path = (start:Node_201511060000 {id:"2519"})-[:CONNECTS_201511060000*]->(end:Node_201511060000 {id:"10026"})
RETURN path, length(path) AS path_length


Neo4j answer is []


Retrying output string
The paths from node 2519 to node 10026 of type node_2 do not exist.


In [21]:
question = """
Return all paths from node 2519 to node 10026 for nodes of type node?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find all paths from node 2519 to node 10026 for nodes of type node?
-----
Cypher Query is MATCH path = (n:Node_201511060000 {id: '2519'})-[:CONNECTS_201511060000*]->(m:Node_201511060000 {id: '10026'}) 
RETURN path


Neo4j answer is []


Retrying output string
The paths between node 2519 and node 10026 for nodes of type node do not exist.


In [22]:
question = """
What are all the paths from node 56203 to node 6939 for nodes of type node?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find all paths from node 56203 to node 6939 where nodes are of type 'node'.
-----
Cypher Query is MATCH path = (a:Node_201511060000 {id: "56203"})-[:CONNECTS_201511060000*]->(b:Node_201511060000 {id: "6939"})
RETURN path


Neo4j answer is []


Whatever you asked for does not exist.


In [23]:
question = """
What is the shortest path from node 2519 to node 10026 for nodes of type node_2?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the shortest path between node 2519 and node 10026 for nodes with type node_2.
-----
Cypher Query is MATCH path=shortestPath((n1:Node_201511060000 {id: '2519'})-[:CONNECTS_201511060000*]-(n2:Node_201511060000 {id: '10026'}))
RETURN path


Neo4j answer is []


Retrying output string
The path between node 2519 and node 10026 does not exist.


In [24]:
question = """
What is the longest existing path for nodes of type node_2?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the longest existing path for nodes of type node_2 in the graph database.
-----
Cypher Query is MATCH path = (:Node_201511060000)-[:CONNECTS_201511060000*]->(:Node_201511060000)
RETURN path
ORDER BY length(path) DESC
LIMIT 1




In [ ]:
question = """
Does a path exist from node 2519 to node 18144 for nodes of type node? If so what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the path between nodes 2519 and 18144 if they are of type 'node'?
-----
Cypher Query is MATCH path = (n:Node{id:'2519'})-[:CONNECTS*..5]->(m:Node{id:'18144'})
RETURN path


Neo4j answer is []


Retrying output string
The path between nodes 2519 and 18144 does not exist.


In [ ]:
question = """
Is node 10026 reachable from node 2519? If so, what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Determine if node 10026 is reachable from node 2519, and provide the path if it is.
-----
Cypher Query is MATCH path=(:Node {id:'2519'})-[:CONNECTS*]->(:Node {id:'10026'}) RETURN path


Neo4j answer is [{'path': [{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS', {'country': 'AU', 'id': '10026', 'org_name': 'Telstra International Limited', 'as_name': 'TELSTRAGLOBAL'}]}]


Yes, node 10026 is reachable from node 2519. The path is through node 3 and node 64.


In [ ]:
question = """
Is node 18144 reachable from node 2519? If so, what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the path, if any, from node 2519 to node 18144.
-----
Cypher Query is MATCH path = (n:Node{id:'2519'})-[:CONNECTS*]->(m:Node{id:'18144'}) RETURN path


Neo4j answer is []


The path from node 2519 to node 18144 does not exist.


In [ ]:
question = """
How many paths traverse through node 10026? For each ending node,consider only one path
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the number of paths that traverse through node 10026, considering only one path for each ending node.
-----
Cypher Query is MATCH (n {id: '10026'})-[r*]->(m)
WITH COUNT(DISTINCT LAST (r)) AS Paths
RETURN Paths


Neo4j answer is [{'Paths': 96}]


Retrying output string
There are no paths that traverse through node 10026.


In [ ]:
question = """
Consider all nodes of node type Node as graph at time t1 and all nodes of type Node_2 as graph at time t2
Between t1 and t2, are there any change in number of paths from node 18144 to node 7670?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the change in number of paths from node 18144 to node 7670 between graph nodes of type Node at time t1 and Node_2 at time t2.
-----
Cypher Query is MATCH p1=(a:Node {id: '18144'})-[:CONNECTS*]->(b:Node {id: '7670'})
WITH count(p1) as paths_t1
MATCH p2=(c:Node_2 {id: '18144'})-[:CONNECTS_2*]->(d:Node_2 {id: '7670'})
RETURN count(p2) - paths_t1 as change_in_paths


Retrying
New Cypher Query is MATCH p1=(a:Node {id: '18144'})-[:CONNECTS*]->(b:Node {id: '7670'})
WITH count(p1) as paths_t1
MATCH p2=(c:Node_2 {id: '18144'})-[:CONNECTS_2*]->(d:Node_2 {id: '7670'})
WITH count(p2) as paths_t2, paths_t1
RETURN paths_t2 - paths_t1 as change_in_paths


Neo4j answer is [{'change_in_paths': 15}]


The change in the number of paths from node 18144 to node 7670 between graph nodes of type Node at time t1 and Node_2 at time t2 is 15.


In [ ]:
question = """
Consider all nodes of node type Node as graph at time t1 and all nodes of type Node_2 as graph at time t2
Between t1 and t2, have the paths from node 2519 to node 10026 changed? If so, how did it change?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the differences in paths between Node 2519 and Node 10026 between t1 and t2 in Graphs of Node and Node_2.
-----
Cypher Query is MATCH path1 = (n:Node {id: '2519'})-[:CONNECTS*]->(m:Node {id: '10026'})
MATCH path2 = (n1:Node_2 {id: '2519'})-[:CONNECTS_2*]->(m1:Node_2 {id: '10026'})
RETURN path1, path2


Neo4j answer is [{'path1': [{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS', {'country': 'AU', 'id': '10026', 'org_name': 'Telstra International Limited', 'as_name': 'TELSTRAGLOBAL'}], 'path2': [{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '225', 'rank': '61', 'id': '1221', 'org_name': 'Telstra Corporation Limited', 'as_name': 'ASN-TELSTRA'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '0', 'rank': '9740', 'id': '38803', 'org_name': 'Wirefreebroadband P

In [ ]:
question = """
Consider all nodes of node type Node as graph at time t1 and all nodes of type Node_2 as graph at time t2
Between t1 and t2, have the paths from node 2519 to node 10026 changed? If so, how did it change?
"""

response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Cypher Query is MATCH p1 = (n1:Node {id:'2519'})-[:CONNECTS*]->(n2:Node {id:'10026'})
WITH collect(p1) AS paths1
MATCH p2 = (n3:Node_2 {id:'2519'})-[:CONNECTS_2*]->(n4:Node_2 {id:'10026'})
WITH paths1, collect(p2) AS paths2
RETURN paths1, paths2, paths1 <> paths2 AS hasChanged


Neo4j answer is [{'paths1': [[{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS', {'country': 'AU', 'id': '10026', 'org_name': 'Telstra International Limited', 'as_name': 'TELSTRAGLOBAL'}]], 'paths2': [[{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '225', 'rank': '61', 'id': '1221', 'org_name': 'Telstra Corporation Limited', 'as_name': 'ASN-TELSTRA'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '0', 'rank': '9740', 'id': '38803', 'org_name': 'Wirefreebroadband Pty Ltd', 'as_name': 'WPL-AS-AP'}, 'CONNECTS_

In [ ]:
question = """
Consider all nodes of node type Node and relationship as CONNECTS as graph at time t1 and
all nodes of type Node_2 and relationship as CONNECTS_2 as graph at time t2
Between t1 and t2, have the paths from node 1136 to node 3333 changed? If so, how did it change?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the changes in paths from node 1136 to node 3333 between graphs with nodes of type Node and relationship CONNECTS at time t1, and nodes of type Node_2 and relationship CONNECTS_2 at time t2.
-----
Cypher Query is MATCH p1=(n:Node {id: "1136"})-[:CONNECTS*..25]->(m:Node {id: "3333"})
WITH nodes(p1) as nodesAtTimeT1
MATCH p2=(n2:Node_2 {id: "1136"})-[:CONNECTS_2*..25]->(m2:Node_2 {id: "3333"})
RETURN nodesAtTimeT1 as nodes_at_Time_T1, nodes(p2) as Nodes_at_Time_T2


Neo4j answer is [{'nodes_at_Time_T1': [{'country': 'NL', 'upstream': '3', 'downstream': '25', 'rank': '108', 'id': '1136', 'org_name': 'KPN B.V.', 'as_name': 'KPN'}, {'upstream': '2', 'downstream': '1', 'rank': '15690', 'id': '3333', 'as_name': 'RIPE-NCC-AS'}], 'Nodes_at_Time_T2': [{'country': 'NL', 'upstream': '3', 'downstream': '25', 'rank': '108', 'id': '1136', 'org_name': 'KPN B.V.', 'as_name': 'KPN'}, {'upstream': '2', 'downstream': '1', 'rank': '15690', 'id': '3333', 'as_name': 'RIPE-NCC-AS'}]}]

In [ ]:
question = """
Consider all nodes of node type Node and relationship as CONNECTS as graph at time t1 and
all nodes of type Node_2 and relationship as CONNECTS_2 as graph at time t2
Between t1 and t2, have the paths from node 1136 to node 3333 changed? If so, how did it change?
"""

response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Cypher Query is MATCH p1=(n1:Node {id:'1136'})-[:CONNECTS*]->(n2:Node {id:'3333'}) 
RETURN p1 AS path_t1
UNION 
MATCH p2=(n3:Node_2 {id:'1136'})-[:CONNECTS_2*]->(n4:Node_2 {id:'3333'}) 
RETURN p2 AS path_t2


Retrying
New Cypher Query is MATCH p1=(n1:Node {id:'1136'})-[:CONNECTS*]->(n2:Node {id:'3333'}) 
RETURN p1 AS path
UNION ALL 
MATCH p2=(n3:Node_2 {id:'1136'})-[:CONNECTS_2*]->(n4:Node_2 {id:'3333'})  
RETURN p2 AS path


Neo4j answer is [{'path': [{'country': 'NL', 'upstream': '3', 'downstream': '25', 'rank': '108', 'id': '1136', 'org_name': 'KPN B.V.', 'as_name': 'KPN'}, 'CONNECTS', {'upstream': '2', 'downstream': '1', 'rank': '15690', 'id': '3333', 'as_name': 'RIPE-NCC-AS'}]}, {'path': [{'country': 'NL', 'upstream': '3', 'downstream': '25', 'rank': '108', 'id': '1136', 'org_name': 'KPN B.V.', 'as_name': 'KPN'}, 'CONNECTS_2', {'upstream': '2', 'downstream': '1', 'rank': '15690', 'id': '3333', 'as_name': 'RIPE-NCC-AS'}]}]


Between time t1 and t2, the paths from node 1136 to node 

In [ ]:
question = """
Are there any isolated nodes of type node_2 in the network? Can you name all?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find all isolated nodes of type node_2 in the network.
-----
Cypher Query is MATCH (n:Node_2)
WHERE NOT (n)-[:CONNECTS_2]-()
RETURN n


Neo4j answer is []


Retrying output string
There are no isolated nodes of type node_2 in the network.


In [ ]:
question = """
Are there any isolated nodes of type node in the network? Can you mention their ids?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find all isolated nodes of type 'node' in the network and list their IDs.
-----
Cypher Query is MATCH (n:Node)
WHERE NOT (n)-[:CONNECTS]-()
RETURN n.id as Isolated_Node_IDs


Neo4j answer is [{'Isolated_Node_IDs': '1'}, {'Isolated_Node_IDs': '2'}]


The isolated nodes of type 'node' in the network are 1 and 2.


In [ ]:
question = """
What is the in degree of node 3333?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the in degree of node 3333.
-----
Cypher Query is MATCH (n:Node{id:"3333"})<-[:CONNECTS]-(c)
RETURN COUNT(c) AS in_degree_of_node_3333


Neo4j answer is [{'in_degree_of_node_3333': 4}]


The in-degree of node 3333 is 4.


In [ ]:
question = """
What is the in degree of node 209?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the in degree of node 209.
-----
Cypher Query is MATCH (n:Node) WHERE n.id="209" RETURN size((n)<--()) as in_degree


Retrying
New Cypher Query is MATCH (n:Node)<--() WHERE n.id="209" RETURN count(*) as in_degree


Neo4j answer is [{'in_degree': 1}]


The in degree of node 209 is 1.


In [ ]:
question = """
What is the in degree of nodes 209 and 3333?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the in-degree of nodes 209 and 3333 in the graph database.
-----
Cypher Query is MATCH (n:Node)
WHERE n.id IN ["209", "3333"]
RETURN n.id AS Node_ID, size((:Node)-[:CONNECTS]->(n)) as In_Degree


Retrying
New Cypher Query is MATCH (n:Node)<-[:CONNECTS]-()
WHERE n.id IN ["209", "3333"]
RETURN n.id AS Node_ID, count(*) as In_Degree


Neo4j answer is [{'Node_ID': '3333', 'In_Degree': 4}, {'Node_ID': '209', 'In_Degree': 1}]


The in-degree of node 209 in the graph database is 1 and the in-degree of node 3333 is 4.


In [ ]:
question = """
What is the in degree of all nodes of type Node?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the in degree of all nodes with type 'Node' in the graph database.
-----
Cypher Query is MATCH (n:Node)
RETURN n.id as Node_ID, size((n)<-[:CONNECTS]-()) as In_Degree


Retrying
New Cypher Query is MATCH (n:Node)<-[:CONNECTS]-()
RETURN n.id as Node_ID, COUNT(*) as In_Degree


Neo4j answer is [{'Node_ID': '42708', 'In_Degree': 1}, {'Node_ID': '50304', 'In_Degree': 4}, {'Node_ID': '3356', 'In_Degree': 5}, {'Node_ID': '45896', 'In_Degree': 4}, {'Node_ID': '8758', 'In_Degree': 3}, {'Node_ID': '6939', 'In_Degree': 2}, {'Node_ID': '4826', 'In_Degree': 1}, {'Node_ID': '38803', 'In_Degree': 1}, {'Node_ID': '2497', 'In_Degree': 2}, {'Node_ID': '4777', 'In_Degree': 3}]


The in degree of all nodes with type 'Node' in the graph database are as follows: Node 42708 has an in degree of 1, Node 50304 has an in degree of 4, Node 3356 has an in degree of 5, Node 45896 has an in degree of 4, Node 8758 has an in degree of 3, Node 6939 has an in degree of 2, Node 4826 has an in de

In [ ]:
question = """
What is the country of node 2914
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the country of node 2914.
-----
Cypher Query is MATCH (n:Node {id:"2914"}) RETURN n.country


Neo4j answer is [{'n.country': 'US'}]


Node 2914 is located in the US.


In [ ]:
question = """
What is the country of node 2915
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the country of node 2915.
-----
Cypher Query is MATCH (n:Node{id:'2915'}) RETURN n.country


Neo4j answer is []


Retrying output string
The country of node 2915 is unknown.


In [ ]:
question = """
What is the highest indegree of all nodes with country name as US
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the highest indegree of all nodes with the attribute 'country name' set to 'US'.
-----
Cypher Query is MATCH (n:Node)<-[:CONNECTS]-()
WHERE n.country = 'US'
RETURN n.as_name AS Node, COUNT(*) AS Indegree
ORDER BY Indegree DESC
LIMIT 1


Neo4j answer is [{'Node': 'LEVEL3', 'Indegree': 5}]


Retrying output string
The node with the highest indegree and the attribute 'country name' set to 'US' is 'LEVEL3', with an indegree of 5.


In [ ]:
### Only error

question = """
What is the highest indegree of all nodes with country as AU
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the node with the highest indegree among all nodes with country = AU.
-----
Cypher Query is MATCH (n:Node {country: 'AU'})
RETURN n, size((:Node)-[:CONNECTS]->(n)) AS indegrees
ORDER BY indegrees DESC
LIMIT 1


Retrying
New Cypher Query is MATCH (n:Node {country: 'AU'})
WITH n, size((:Node)-[:CONNECTS]->n) AS indegrees
ORDER BY indegrees DESC
LIMIT 1
RETURN n


Retrying output string
The node with the highest indegree among all nodes with country = AU does not exist.


## For output

In [ ]:
question = """
Does a path exist from node 3356 to node 7018 for nodes of type node_2? If so what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find a path from node 3356 to node 7018 for nodes of type node_2.
-----
Cypher Query is MATCH path = (n:Node_2 {id:'3356'})-[:CONNECTS_2*]->(m:Node_2 {id:'7018'}) 
RETURN path


Neo4j answer is [{'path': [{'country': 'US', 'upstream': '26', 'city': 'Monroe', 'downstream': '5281', 'rank': '15', 'id': '3356', 'state': 'LA', 'org_name': 'Level 3 Parent, LLC', 'as_name': 'LEVEL3'}, 'CONNECTS_2', {'upstream': '10', 'downstream': '1660', 'rank': '110', 'id': '3257', 'as_name': 'GTT-BACKBONE'}, 'CONNECTS_2', {'country': 'US', 'upstream': '7', 'city': 'MIDDLETOWN', 'downstream': '1592', 'rank': '2', 'id': '7018', 'state': 'NJ', 'org_name': 'AT&T Services, Inc.', 'as_name': 'ATT-INTERNET4'}]}, {'path': [{'country': 'US', 'upstream': '26', 'city': 'Monroe', 'downstream': '5281', 'rank': '15', 'id': '3356', 'state': 'LA', 'org_name': 'Level 3 Parent, LLC', 'as_name': 'LEVEL3'}, 'CONNECTS_2', {'country': 'US', 'upstream': '7', 'city': 'MIDDLETOWN', 'downstream': '1592', 'rank':

In [ ]:
question = """
Does a path exist from node 123 to node 7671 for nodes of type node_2? If so what is the path?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the path from node 123 to node 7671 for nodes of type node_2.
-----
Cypher Query is MATCH path = (a:Node_2 {id: '123'})-[:CONNECTS_2*]->(b:Node_2 {id: '7671'})
RETURN path


Neo4j answer is []


Retrying output string
The path from node 123 to node 7671 for nodes of type node_2 does not exist.


In [ ]:
question = """
What is the shortest path from node 2519 to node 10026 for nodes of type node_2?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the shortest path from node 2519 to node 10026 for nodes of type node_2.
-----
Cypher Query is MATCH path = shortestPath((n:Node_2 {id: "2519"})-[:CONNECTS_2*]-(m:Node_2 {id: "10026"}))
RETURN path


Neo4j answer is [{'path': [{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '225', 'rank': '61', 'id': '1221', 'org_name': 'Telstra Corporation Limited', 'as_name': 'ASN-TELSTRA'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '3', 'downstream': '193', 'rank': '3029', 'id': '4608', 'org_name': 'Asia Pacific Network Information Centre', 'as_name': 'APNIC-SERVICES'}, 'CONNECTS_2', {'country': 'AU', 'id': '10026', 'org_name': 'Telstra International Limited', 'as_name': 'TELSTRAGLOBAL'}]}]


The shortest path from node 2519 to node 10026 of type node_2 is as follows: 2519 (VECTANT) connects to 1221 (ASN-TELSTRA), which then connects to 4608 (APNIC-SERV

In [ ]:
question = """
Consider all nodes of node type Node and relationship as CONNECTS as graph at time t1 and
all nodes of type Node_2 and relationship as CONNECTS_2 as graph at time t2
Between t1 and t2, are there any change in number of paths from node 18144 to node 7670?
If so what is the change?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the change in the number of paths from node 18144 to node 7670 between time t1 (when nodes of type Node and relationships of type CONNECTS are present in the graph) and time t2 (when nodes of type Node_2 and relationships of type CONNECTS_2 are present in the graph).
-----
Cypher Query is MATCH p1 = (n1:Node {id: "18144"})-[:CONNECTS*]->(n2:Node {id: "7670"})
WITH count(p1) as paths_t1
MATCH p2 = (m1:Node_2 {id: "18144"})-[:CONNECTS_2*]->(m2:Node_2 {id: "7670"})
WITH count(p2) as paths_t2, paths_t1
RETURN (paths_t2 - paths_t1) as change_in_paths


Neo4j answer is [{'change_in_paths': 15}]


The change in the number of paths from node 18144 to node 7670 between time t1 and t2 is 15.


In [ ]:
question = """
Consider all nodes of node type Node and relationship as CONNECTS as graph at time t1 and
all nodes of type Node_2 and relationship as CONNECTS_2 as graph at time t2
Between t1 and t2, have the paths from node 1136 to node 3333 changed? If so, how did it change?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find any changes in the paths from node 1136 to node 3333 that occurred between time t1 and t2 in the graph database consisting of nodes of type Node and relationship CONNECTS at time t1 and nodes of type Node_2 and relationship CONNECTS_2 at time t2.
-----
Cypher Query is MATCH path1 = (x:Node {id: '1136'})-[:CONNECTS*]->(y:Node {id: '3333'})
WITH collect(nodes(path1)) as nodes1
MATCH path2 = (a:Node_2 {id: '1136'})-[:CONNECTS_2*]->(b:Node_2 {id: '3333'})
WITH nodes1, collect(nodes(path2)) as nodes2
RETURN nodes1, nodes2


Neo4j answer is [{'nodes1': [[{'country': 'NL', 'upstream': '3', 'downstream': '25', 'rank': '108', 'id': '1136', 'org_name': 'KPN B.V.', 'as_name': 'KPN'}, {'upstream': '2', 'downstream': '1', 'rank': '15690', 'id': '3333', 'as_name': 'RIPE-NCC-AS'}]], 'nodes2': [[{'country': 'NL', 'upstream': '3', 'downstream': '25', 'rank': '108', 'id': '1136', 'org_name': 'KPN B.V.', 'as_name': 'KPN'}, {'upstream': '2', 'downstream': '1', 'rank': '15690', 'id

In [ ]:
question = """
Are there any isolated nodes of type node_2 in the network? Can you name all?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find all isolated nodes with type node_2 in the network; list them.
-----
Cypher Query is MATCH (n:Node_2)
WHERE NOT (n)-[:CONNECTS_2]-()
RETURN n


Neo4j answer is []


Retrying output string
There are no isolated nodes with type node_2 in the network.


In [ ]:
question = """
Are there any isolated nodes of type Node in the network? Can you name all?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find any isolated nodes of type Node in the network and list all names.
-----
Cypher Query is MATCH (n:Node)
WHERE NOT (n)-[:CONNECTS]-()
RETURN n.as_name as IsolatedNodeNames


Neo4j answer is [{'IsolatedNodeNames': 'LVLT-1'}, {'IsolatedNodeNames': 'UDEL-DCN'}]


The isolated nodes in the network are LVLT-1 and UDEL-DCN.


In [ ]:
question = """
What is the highest indegree of all nodes with country as AU
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the highest indegree of all nodes with country equal to 'AU'.
-----
Cypher Query is MATCH (n:Node)<-[:CONNECTS]-()
WHERE n.country = 'AU'
RETURN n.id, COUNT(*) AS indegree
ORDER BY indegree DESC
LIMIT 1


Neo4j answer is [{'n.id': '45896', 'indegree': 4}]


The highest indegree of all nodes with country equal to 'AU' is 4.


In [ ]:
question = """
What is the highest indegree of all nodes with country as AU
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the highest indegree of all nodes with country as AU.
-----
Cypher Query is MATCH (n:Node) WHERE n.country = 'AU'
RETURN n.id, SIZE((:Node)-[:CONNECTS]->(n)) as indegree
ORDER BY indegree DESC
LIMIT 1


Retrying
New Cypher Query is MATCH (n:Node) WHERE n.country = 'AU'
RETURN n.id, SIZE(()-[:CONNECTS]->(n)) as indegree
ORDER BY indegree DESC
LIMIT 1


Whatever you asked for does not exist.


In [ ]:
question = """
What is the in degree of nodes 209 and 3333?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the in degree of nodes 209 and 3333.
-----
Cypher Query is MATCH (n:Node)
WHERE n.id IN ['209', '3333']
RETURN n.id AS Node, SIZE(()-[:CONNECTS]->(n)) AS InDegree


Retrying
New Cypher Query is MATCH (n:Node)<-[:CONNECTS]-()
WHERE n.id IN ['209', '3333']
RETURN n.id AS Node, COUNT(*) AS InDegree


Neo4j answer is [{'Node': '3333', 'InDegree': 4}, {'Node': '209', 'InDegree': 1}]


The in-degree of node 209 is 1, and the in-degree of node 3333 is 4.


In [ ]:
question = """
What is the in degree of nodes 209 and 3333?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the in degree of nodes 209 and 3333.
-----
Cypher Query is MATCH (n:Node)
WHERE n.id IN ['209', '3333']
RETURN n.id, SIZE(()-[:CONNECTS]->(n)) as inDegree


Retrying
New Cypher Query is MATCH (n:Node)<-[:CONNECTS]-()
WHERE n.id IN ['209', '3333']
RETURN n.id as nodeId, COUNT(*) as inDegree


Neo4j answer is [{'nodeId': '3333', 'inDegree': 4}, {'nodeId': '209', 'inDegree': 1}]


The in degree of node 209 is 1, and for node 3333, it is 4.


In [ ]:
question = """
Consider all nodes of node type Node as graph at time t1 and all nodes of type Node_2 as graph at time t2
Between t1 and t2, have the paths from node 2519 to node 10026 changed? If so, how did it change?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the changes in the paths from node 2519 to node 10026 between time t1 and t2, considering all nodes of type Node as graph at time t1 and all nodes of type Node_2 as graph at time t2.
-----
Cypher Query is MATCH path1 = (n1:Node{ id: '2519' })-[:CONNECTS*]->(n2:Node{ id: '10026' })
WITH collect(path1) AS paths_t1
MATCH path2 = (n1:Node_2{ id: '2519' })-[:CONNECTS_2*]->(n2:Node_2{ id: '10026' })
WITH paths_t1, collect(path2) AS paths_t2
RETURN paths_t1 AS Paths_at_t1, paths_t2 AS Paths_at_t2


Neo4j answer is [{'Paths_at_t1': [[{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS', {'country': 'AU', 'id': '10026', 'org_name': 'Telstra International Limited', 'as_name': 'TELSTRAGLOBAL'}]], 'Paths_at_t2': [[{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '225', 'rank': '61', 'id': '

In [ ]:
question = """
Consider all nodes of node type Node as graph at time t1 and all nodes of type Node_2 as graph at time t2
Between t1 and t2, have the paths from node 2519 to node 10026 changed? If so, how did it change?
"""
question = refine_prompt(question)["updated_user_prompt"]
print("Refined user input:", question)
print("-----")
response = lang_chain_custom(question,driver,model='gpt-4')
print(response)

Refined user input: Find the changes in paths from node 2519 to node 10026 between graph of node type Node at time t1 and graph of node type Node_2 at time t2.
-----
Cypher Query is MATCH p1=(n1:Node {id:'2519'})-[:CONNECTS*]->(m1:Node {id:'10026'})
MATCH p2=(n2:Node_2 {id:'2519'})-[:CONNECTS_2*]->(m2:Node_2 {id:'10026'})
RETURN p1 AS Path_at_T1, p2 AS Path_at_T2


Neo4j answer is [{'Path_at_T1': [{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS', {'country': 'AU', 'id': '10026', 'org_name': 'Telstra International Limited', 'as_name': 'TELSTRAGLOBAL'}], 'Path_at_T2': [{'country': 'JP', 'upstream': '3', 'downstream': '64', 'rank': '314', 'id': '2519', 'as_name': 'VECTANT'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '225', 'rank': '61', 'id': '1221', 'org_name': 'Telstra Corporation Limited', 'as_name': 'ASN-TELSTRA'}, 'CONNECTS_2', {'country': 'AU', 'upstream': '2', 'downstream': '0', 'rank': '9740'